In [ ]:
import pathlib
import requests
import urllib
import dotenv
import os
import shapely
import shapely.geometry
import geopandas

In [ ]:
SCHEME = "https"
NETLOC_API = "data.linz.govt.nz"
WFS_PATH_API_START = "/services;key="
WFS_PATH_API_END = "/wfs"

dotenv.load_dotenv()
KEY = os.environ.get('LINZ_API', None)

CRS = "EPSG:2193"

data_url = urllib.parse.urlunparse((SCHEME, NETLOC_API, f"{WFS_PATH_API_START}{KEY}{WFS_PATH_API_END}", "", "", ""))

In [ ]:
KEY

In [ ]:
x0 = 1477354
x1 = 1484656
y0 = 5374408
y1 = 5383411

In [ ]:
def make_api_params(layer, geometry_type):
    api_query = {
                "service": "WFS",
                "version": 2.0,
                "request": "GetFeature",
                "typeNames": f"layer-{layer}",
                "outputFormat": "json",
                "SRSName": f"{CRS}",
                "cql_filter": f"bbox({geometry_type}, {y0}, {x0}, " +
                              f"{y1}, {x1}, " +
                              f"'urn:ogc:def:crs:{CRS}')"
            }
    return api_query

https://data.linz.govt.nz/services/query/v1/vector.json?key=b74b510da6cb4e23ad8e779f9bd1b366&layer=105448&x=171.99059591063107&y=-41.78711137951029&max_results=3&radius=10000&geometry=true&with_field_names=true

## NZ coastline

In [ ]:
layer = 51153
geom_type = "GEOMETRY"

params = make_api_params(layer, geom_type)

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()
requests.Request('POST', data_url, params=params).prepare().url

In [ ]:
json_response

In [ ]:
json_response.keys()

In [ ]:
print(json_response['features'][0].keys())
[json_response['features'][0]['type'], json_response['features'][0]['id'], json_response['features'][0]['geometry_name'], 
 json_response['features'][0]['properties']]

In [ ]:
response.headers['Content-Type']

# Bathymetry contours

In [ ]:
layer = 50448
geom_type = "shape"
params = make_api_params(layer, geom_type)

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()
requests.Request('POST', data_url, params=params).prepare().url

In [ ]:
json_response

In [ ]:
response.headers

In [ ]:
response.headers['Content-Type'] == 'application/json;charset=utf-8'

# Overcome crash if invalid query but status is 200 and then try requests.json
For instance if we have the incorrect cql_filter bbox label (i.e. "GEOMETRY" when it should be "shape", we get a status=200, but we get a crash if requests.json().
Below is an example of how to detect if the URL response is invalid. We could use to try 'shape' then 'GEOMETRY'

In [ ]:
layer = 50448
geom_type = "GEOMETRY"
params = make_api_params(layer, geom_type)

response = requests.get(data_url, params=params, stream=True)
requests.Request('POST', data_url, params=params).prepare().url

In [ ]:
response.raise_for_status()

In [ ]:
response.headers

In [ ]:
response.text

In [ ]:
'Exception' in response.text

# Create tests for no boundary filter

In [ ]:
def make_api_params_no_bounds(layer):
    api_query = {
                "service": "WFS",
                "version": 2.0,
                "request": "GetFeature",
                "typeNames": f"layer-{layer}",
                "outputFormat": "json",
                "SRSName": f"{CRS}"
            }
    return api_query

## Railway centre lines

In [ ]:
crs=2193
params = make_api_params_no_bounds(50781)
response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()

In [ ]:
features = {'geometry': []}
for feature in json_response['features']:

    shapely_geometry = shapely.geometry.shape(feature['geometry'])

    # Create column headings for each 'properties' option from the first in-bounds vector
    if len(features['geometry']) == 0:
        for key in feature['properties'].keys():
            features[key] = []  # The empty list to append the property values too

    # Convert any one Polygon MultiPolygon to a straight Polygon then add to the geometries
    if (shapely_geometry.geometryType() == 'MultiPolygon' and len(shapely_geometry) == 1):
        shapely_geometry = shapely_geometry[0]
    features['geometry'].append(shapely_geometry)

    # Add the value of each property in turn
    for key in feature['properties'].keys():
        features[key].append(feature['properties'][key])

# Convert to a geopandas dataframe
if len(features) > 0:
    features = geopandas.GeoDataFrame(features, crs=crs)
else:
    features = None

In [ ]:
features.geometry.length.sum()

In [ ]:
list(features['id'][0:5])

## Pastural land

In [ ]:
crs=2193
params = make_api_params_no_bounds(51572)
response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()

In [ ]:
features = {'geometry': []}
for feature in json_response['features']:

    shapely_geometry = shapely.geometry.shape(feature['geometry'])

    # Create column headings for each 'properties' option from the first in-bounds vector
    if len(features['geometry']) == 0:
        for key in feature['properties'].keys():
            features[key] = []  # The empty list to append the property values too

    # Convert any one Polygon MultiPolygon to a straight Polygon then add to the geometries
    if (shapely_geometry.geometryType() == 'MultiPolygon' and len(shapely_geometry) == 1):
        shapely_geometry = shapely_geometry[0]
    features['geometry'].append(shapely_geometry)

    # Add the value of each property in turn
    for key in feature['properties'].keys():
        features[key].append(feature['properties'][key])

# Convert to a geopandas dataframe
if len(features) > 0:
    features = geopandas.GeoDataFrame(features, crs=crs)
else:
    features = None

In [ ]:
features.loc[0].geometry.geometryType()

In [ ]:
list(features['id'][0:5])

In [ ]:
features.columns

In [ ]:
features.geometry.area.sum()

In [ ]:
features.geometry.length.sum()